# **CRNN for Image to Text**

**Importing necesary libraries**

In [ ]:
import tensorflow as tf
import keras.backend as K
import numpy as np
import matplotlib.image as mpimg
import os

# **Getting necessary data**
All the images and text were filtered and sorted in two folders, many duplicate files were deleted and labeling of the data was checked and matched

In [ ]:
img_path = '/content/drive/My Drive/DATA ML/0325updated.task2train(626p)/images'
txt_path = '/content/drive/My Drive/DATA ML/0325updated.task2train(626p)/text'

#**Image Preprocessing**
The images present in the dataset had different sizes and different channels, to tackle that, maximum length and width were taken by iterating on all images.

All the images were padded to the same length and width and converted to grayscale.

Due to the size of the images being large, images were saved to the hard drive and can be accessed whenever needed

In [ ]:
##IMAGE PREPROCESSING ##
def preprocess_image(path):
  import tensorflow
  import os
  import cv2
  import matplotlib.image
  from tqdm.auto import tqdm
  cwd = os.getcwd()
  preprocess_image.w = 0
  preprocess_image.h = 0
  num = 0
  total = 0
  for image_path in os.listdir(path):
    total +=1
  directory = str(cwd)+'/training_prep_img'
  os.mkdir(directory)
  loop = tqdm(total=total, position=0, leave=False)
  for k in range(total):
    loop.set_description("Finding Maximum Width and Height...".format(k))
    #Finding the maximum length and width of the image
  for image_path in os.listdir(path):
    input_path = os.path.join(path, image_path)
    img = matplotlib.image.imread(input_path)
    if len(img.shape) == 3:
      a, b, c = img.shape
    elif len(img.shape) == 2:
      a, b = img.shape  
    if a > preprocess_image.w:
      preprocess_image.w = a
    if b > preprocess_image.h:
      preprocess_image.h = b  
    else:
      pass
    loop.update(1)
  print('Maximum Width: ' +str(preprocess_image.w))
  print('Maximum Height: ' +str(preprocess_image.h))
  loop2 = tqdm(total=total, position=0, leave=False)
  for m in range(total):
    loop2.set_description("Preprocessing Images...".format(m))
    #preprocessing the images to pad them to maximum length and width and converted to grayscale
  for image_path in os.listdir(path):
    input_path = os.path.join(path, image_path)
    img = matplotlib.image.imread(input_path)

    if len(img.shape) == 3:
      a, b, c = img.shape
    elif len(img.shape) == 2:
      a, b = img.shape
      c = 0

    if a == preprocess_image.w and b == preprocess_image.h:
      if c <= 1:
        img2numpy = tensorflow.keras.preprocessing.image.img_to_array(img, data_format='channels_last', dtype=None)
      else:
        gray_img = tensorflow.image.rgb_to_grayscale(img, name=None)
        std_img = tensorflow.image.per_image_standardization(gray_img)
        img2numpy = tensorflow.keras.preprocessing.image.img_to_array(std_img, data_format='channels_last', dtype=None)

    else:
      gray_img = tensorflow.image.rgb_to_grayscale(img, name=None)
      padded_img = tensorflow.image.resize_with_pad(gray_img, preprocess_image.h, preprocess_image.w, method='bicubic', antialias=False)
      std_img = tensorflow.image.per_image_standardization(padded_img)
      img2numpy = tensorflow.keras.preprocessing.image.img_to_array(std_img, data_format='channels_last', dtype=None)
    num += 1
 
    filename = image_path
    os.chdir(directory)
    img = cv2.convertScaleAbs(img2numpy, alpha=(255.0))
    cv2.imwrite(filename, img)
    #print('Number of files Preprocessed : '+str(num)) 
    os.chdir(cwd)
    loop2.update(1)

In [ ]:
preprocess_image(img_path)

Maximum Width: 7016
Maximum Height: 4961


In [ ]:
w = preprocess_image.w
h = preprocess_image.h

#**Text Preprocessing**
The text is first saved to an array and then processed over TensorFlow text preprocessing library, by Tokenizing, fitting text to sequences and then padding the texts.

In [ ]:
text_data = []
def preprocess_text(path):
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  import tensorflow
  import numpy as np
  from tqdm.auto import tqdm
  total = 0
  for m in os.listdir(path):
    total +=1
  loop = tqdm(total=total, position=0, leave=False)
  #appending text to an array
  for k in range(total):
    loop.set_description("Saving data to array for preprocessing...".format(k))
  for text_path in os.listdir(path):
    input_path_text = os.path.join(path, text_path)
    with open (input_path_text, "r") as myfile:
                  data=myfile.readlines()
                  text_data.append(data)
    loop.update(1)
  #preprocessing the txt
  print('Processing Text.....')                  
  tokenizer = tensorflow.keras.preprocessing.text.Tokenizer( num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token="<OOV>")
  tokenizer.fit_on_texts(text_data)
  seq = tokenizer.texts_to_sequences(text_data)
  preprocess_text.padded = pad_sequences(seq, padding='post')
  print('Processing Completed')

In [ ]:
preprocess_text(txt_path)

Processing Text.....
Processing Completed


In [ ]:
padded = preprocess_text.padded

#**Custom Generator**
This generator takes in the image files and text to process them and  pairing the images and texts that is then called by our custom generator class that inturn helps in sending the data to model in batches so as to not have an OOM state.

In [ ]:
os.chdir('/content')
cwd = os.getcwd()
directory = str(cwd)+'/training_prep_img'
filenames = []
def sequencing_data(path):
  import os
  import shutil
  import numpy as np
  for subdir, dirs, files in os.walk(path):
    for file in files:
        filenames.append(file)
  os.chdir(path)
  np.save('filenames.npy', filenames)
  np.save('padded.npy', padded)

  from sklearn.utils import shuffle
# Saving the name of files to be accessed later
  filenames_shuffled, padded_shuffled = shuffle(filenames, padded)
  np.save('padded_shuffled.npy', padded_shuffled)
  np.save('filenames_shuffled.npy', filenames_shuffled)

  from sklearn.model_selection import train_test_split
  filenames_shuffled_numpy = np.array(filenames_shuffled)

  sequencing_data.X_train_filenames, sequencing_data.X_val_filenames, sequencing_data.y_train, sequencing_data.y_val = train_test_split(filenames_shuffled_numpy, padded_shuffled, test_size=0.2, random_state=1)

# You can save these files as well. As you will be using them later for training and validation of your model.
  np.save('X_train_filenames.npy', sequencing_data.X_train_filenames)
  np.save('y_train.npy', sequencing_data.y_train)

  np.save('X_val_filenames.npy', sequencing_data.X_val_filenames)
  np.save('y_val.npy', sequencing_data.y_val)

In [ ]:
sequencing_data(directory)

In [ ]:
X_train_filenames = sequencing_data.X_train_filenames
X_val_filenames = sequencing_data.X_val_filenames
y_train = sequencing_data.y_train
y_val = sequencing_data.y_val

In [ ]:
from skimage.transform import resize
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    return np.array([
                 resize(mpimg.imread(str(directory)+'/'+ str(file_name)), (w, h, 1))
                  for file_name in batch_x])/255.0, np.array(batch_y)

In [ ]:
batch_size = 16

my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size)

#**MODEL**
This model is based on the research paper *An End-to-End Trainable Neural Network for Image-based Sequence
Recognition and Its Application to Scene Text Recognition* by *Baoguang Shi, Xiang Bai and Cong Yao*. The model is not exactly copied but tailored to out application here.

The model architecture made by Baoguang Shi, Xiang Bai and Cong Yao is based on the VGG- Very Deep architectures.

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(4961 ,7016, 1)),
  tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=3),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=3),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=3),
  tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
  tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2, 1), strides=3),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(512, (3,3), activation = 'relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2, 1), strides=3),
  tf.keras.layers.Conv2D(512, (3,3), activation = 'relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=3),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPool2D(pool_size=(2, 1), strides=3),
  tf.keras.layers.Conv2D(512, (2,2), activation = 'relu'),
  tf.keras.layers.Lambda(lambda x: K.squeeze(x, 1)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout = 0.2)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout = 0.2)),
  tf.keras.layers.Dense(128, activation='relu')
])
model.summary()

In [ ]:
model.compile(optimizer = 'Adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
model.fit(my_training_batch_generator,
                   epochs = 100,
                   verbose = 1,
                   validation_data = my_validation_batch_generator)